In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
train = pd.read_csv('../MNIST_data/train.csv')

In [3]:
train_x = (train.iloc[:, 1:] - 128) / 256

In [4]:
train_y_1d = train.iloc[:, 0]

In [5]:
train_y = np.zeros((train_x.shape[0], 10))

In [6]:
train_y[list(range(train_x.shape[0])), train_y_1d] = 1

In [7]:
train_y[:5]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [8]:
train_y_1d.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

In [9]:
x = tf.placeholder(tf.float32, shape=[None, 784], name='input_x')
y = tf.placeholder(tf.float32, shape=[None, 10], name='label_y')

In [10]:
with tf.name_scope('layer1'):
    w1 = tf.Variable(tf.random_normal([784, 50], mean=0, stddev=1))
    b1 = tf.Variable(tf.zeros([50]))
    layer1 = tf.sigmoid(tf.matmul(x, w1) + b1)
with tf.name_scope('layer2'):
    w2 = tf.Variable(tf.random_normal([50, 10], mean=0, stddev=1))
    b2= tf.Variable(tf.zeros([10]))
    y_ = tf.sigmoid(tf.matmul(layer1, w2) + b2, name='logit')

In [11]:
with tf.name_scope('xent'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
    tf.summary.scalar('cross_entropies', cross_entropy)

In [12]:
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy, name='train_step')

In [13]:
with tf.name_scope('accurary'):
    correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    tf.summary.scalar('accuracies', accuracy)

In [14]:
with tf.name_scope('prediction'):
    pred = tf.argmax(y_, 1, name='pred')

In [15]:
writer = tf.summary.FileWriter('/tmp/mnist/3')

In [16]:
init = tf.global_variables_initializer()

In [17]:
sess = tf.Session()

In [18]:
writer.add_graph(sess.graph)

In [19]:
row_ids = list(range(train_x.shape[0] - 2000))

In [20]:
batch_size = 1000

In [21]:
sess.run(init)

In [22]:
merged_summaries = tf.summary.merge_all()

In [23]:
for _ in range(50):
    batch_row_ids = np.random.choice(row_ids, size=batch_size, replace=False)
    train_step.run(session=sess, feed_dict={x: train_x.iloc[batch_row_ids], y: train_y[batch_row_ids, :]})
    summaries = sess.run(merged_summaries, feed_dict={x: train_x.iloc[batch_row_ids], y: train_y[batch_row_ids, :]})
    writer.add_summary(summaries, _)

In [24]:
sess.close()